In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import pickle
import os
from tqdm import tqdm
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
import numpy as np
import collections
from gensim.models import KeyedVectors
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(1)

In [2]:
data_df = pd.read_csv('Question_Classification_Dataset.csv')

In [3]:
data_df.head()

,Unnamed: 0,Questions,Category0,Category1,Category2
0,0,How did serfdom develop in and then leave Russ...,DESCRIPTION,DESC,manner
1,1,What films featured the character Popeye Doyle ?,ENTITY,ENTY,cremat
2,2,How can I find a list of celebrities ' real na...,DESCRIPTION,DESC,manner
3,3,What fowl grabs the spotlight after the Chines...,ENTITY,ENTY,animal
4,4,What is the full form of .com ?,ABBREVIATION,ABBR,exp


In [4]:
le = preprocessing.LabelEncoder()
data_texts = data_df['Questions'].to_list()
labels = le.fit_transform(data_df['Category0'])

In [5]:

class_num=len(data_df['Category0'].unique())
print(class_num)
print(len(labels)) # rows =5452

6
5452


In [6]:
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [10]:
#model['hello']
print(model['1'].shape)

(300,)


In [11]:
import re

def getRidOfXXX(data_texts,model):
    new_one=[]
    for text in data_texts:
        #_text=re.sub("[':!`\?,\.\)\(]",'',text)
        word_list=text.split()
        
        for word in word_list:
            if word not in model:
                print(word)
                word_list.remove(word)
        new_text=' '.join(word_list)
        new_one.append(new_text)
    return new_one

data_texts=getRidOfXXX(data_texts,model)


In [12]:
print(data_texts[0])

How did serfdom develop in and then leave Russia 


In [13]:
def getMaxLen(data_texts):
    max=0
    for text in data_texts:
        if len(text.split()) > max:
            max=len(text.split())
    return max

max_len=getMaxLen(data_texts)
print(max_len)

33


In [19]:
def padding(data_texts,max_len):
    new_data=[]
    for text in data_texts:
        delta=max_len - len(text.split())
        #print(delta)
        new_text='<PAD> '*delta+text
        new_data.append(new_text)
    return new_data

data_texts=padding(data_texts,max_len)
print(data_texts[0])

<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> How did serfdom develop in and then leave Russia 


In [26]:
model['<PAD>']=np.array([0]*300)

In [29]:
EMBEDDING_SIZE = model['<PAD>'].shape[0]


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [ ]:
def encodingTexts(data_texts,model):
  data=[]
  for text in data_texts:
    encoded_texts=[]
    word_list=text.split()
    for word in word_list:
      encoded_texts.append(model[word])
    data.append(encoded_texts)

data=encodingTexts(data_texts,model)

#data=model(data_texts)
#data=torch.Tensor(data)
#targets=torch.Tensor(targets)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_texts, labels, test_size=0.2, random_state=0)

In [ ]:
BATCH_SIZE = 4
BATCH_NUM = len(labels)//BATCH_SIZE
LR=0.01
EPOCH_NUM=10


for epoch in range(EPOCH_NUM):
  for batch in range(BATCH_NUM):
    for i in range(5):
      
 ff